In [1]:
from classifiers import ConformalPredictor, WCRF, vanillaRF
from data import german_credit_data
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from eval import conformal_pred_eval, wcrf_eval
import numpy as np


## Load pre-processed data

In [2]:
## Eventually, for data in datasets...

X_train, X_calib,X_test, y_train, y_calib, y_test = german_credit_data()

## Initialize models
TODO - hyper-parameter tuning

In [3]:
rfc = vanillaRF(X_train, y_train)
# Conformal RF
cpc = ConformalPredictor()
# Weighted Cautiouse Random Forest
wcrf = WCRF(s=1,gamma=1,labda=5)

## Fit models

In [4]:
# Vanilla Random Forest
best_params = rfc.gridSearch()
rfc.fit(best_params)
# Conformal RF
cpc.fit(X_train, y_train, X_calib, y_calib)
# Weighted Cautiouse Random Forest
wcrf.fit(X_train, y_train)
wcrf.fit_w(X_train, y_train)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_

## Run Eval

In [5]:
# Vanilla Random Forest
print(accuracy_score(y_test, rfc.predict(X_test)))
# Conformal RF
cpc_preds = cpc.predict(X_test)
cpc_pred_transformed = [
    -1 if p == (0, 1) else p[0] 
    for p in cpc_preds
]
print(conformal_pred_eval(y_test, np.array(cpc_pred_transformed), cpc))
# Weighted Cautiouse Random Forest
print(wcrf_eval(X_test,y_test, wcrf))



0.74
{'u65_score': 71.05, 'single_set_accuracy': 91.3, 'determinacy': 23.0}
{'u65_score': 78.0, 'single_set_accuracy': 78.0, 'determinacy': 100.0, 'precise_accuracy': 77.0}
